In [1]:
# 自动计算cell的计算时间
%load_ext autotime

%matplotlib inline
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

%reload_ext tensorboard

time: 424 ms (started: 2021-09-01 13:27:09 +08:00)


In [6]:
from tensorboard import notebook
notebook.list()

No known TensorBoard instances running.
time: 2.35 ms (started: 2021-09-01 00:17:12 +08:00)


In [ ]:
notebook.start("--logdir ./lightning_logs/")

In [3]:
%%bash

# 增加更新
git add *.ipynb */*.ipynb

git remote -v

git commit -m '更新 #4 Aug 31, 2021'

#git push origin master
git push

origin	git@github.com:ustchope/pytorch_lightning_study.git (fetch)
origin	git@github.com:ustchope/pytorch_lightning_study.git (push)
[main 1a4dd6f] 更新 #4 Aug 31, 2021
 1 file changed, 97 insertions(+), 74 deletions(-)


To github.com:ustchope/pytorch_lightning_study.git
   cf31512..1a4dd6f  main -> main


time: 3.81 s (started: 2021-08-31 23:42:49 +08:00)


In [7]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
from pytorch_lightning import LightningModule, Trainer

time: 1.64 ms (started: 2021-09-01 13:28:27 +08:00)


本指南将带您了解 PyTorch Lightning 的核心部分。

我们将完成以下工作：
* 实现一个 MNIST 分类器。
* 使用继承实现 AutoEncoder

> 任何 DL/ML PyTorch 项目都适合 Lightning 结构。 在这里，我们只专注于 3 种类型的研究来说明。

# 从 MNIST 到自动编码器

## 研究代码

### 模型

闪电模块包含所有核心研究成分：
* 该模型
* 优化器
* 训练/验证/测试步骤

让我们首先从模型开始。 在这种情况下，我们将设计一个 3 层神经网络。

In [3]:
import torch
from torch.nn import functional as F
from torch import nn
from pytorch_lightning.core.lightning import LightningModule


class LitMNIST(LightningModule):
    def __init__(self):
        super().__init__()

        # mnist images are (1, 28, 28) (channels, height, width)
        self.layer_1 = nn.Linear(28 * 28, 128)
        self.layer_2 = nn.Linear(128, 256)
        self.layer_3 = nn.Linear(256, 10)

    def forward(self, x):
        batch_size, channels, height, width = x.size()

        # (b, 1, 28, 28) -> (b, 1*28*28)
        x = x.view(batch_size, -1)
        x = self.layer_1(x)
        x = F.relu(x)
        x = self.layer_2(x)
        x = F.relu(x)
        x = self.layer_3(x)

        x = F.log_softmax(x, dim=1)
        return x

time: 2.21 ms (started: 2021-09-01 13:27:22 +08:00)


请注意，这是一个闪电模块，而不是 torch.nn.Module。 LightningModule 等效于纯 PyTorch 模块，只是它增加了功能。 但是，您可以像使用 PyTorch 模块一样使用它。

In [8]:
net = LitMNIST()
x = torch.randn(1, 1, 28, 28)
out = net(x)

time: 57.6 ms (started: 2021-08-31 21:54:52 +08:00)


In [9]:
out

tensor([[-2.3127, -2.2126, -2.3196, -2.2438, -2.2116, -2.2051, -2.3613, -2.3832,
         -2.3733, -2.4333]], grad_fn=<LogSoftmaxBackward>)

time: 6.66 ms (started: 2021-08-31 21:55:31 +08:00)


现在我们添加了包含所有训练循环逻辑的 training_step

In [ ]:
class LitMNIST(LightningModule):
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        return loss

### 数据

闪电在纯数据加载器上运行。 这是用于加载 MNIST 的 PyTorch 代码。

In [5]:
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST
import os
from torchvision import datasets, transforms

# transforms
# prepare transforms standard to MNIST
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

# data
mnist_train = MNIST(os.getcwd(), train=True, download=True, transform=transform)
mnist_train = DataLoader(mnist_train, batch_size=64, num_workers = 24)

time: 47.8 ms (started: 2021-09-01 13:27:29 +08:00)


您可以通过 3 种方式使用 DataLoaders：

1. 将 DataLoaders 传递给 .fit()

将数据加载器传递给 .fit() 函数。

In [ ]:
model = LitMNIST()
trainer = Trainer()
trainer.fit(model, mnist_train)

2. LightningModule 数据加载器

对于快速的研究原型设计，将模型与数据加载器链接可能更容易。

In [13]:
class LitMNIST(pl.LightningModule):
    def train_dataloader(self):
        # transforms
        # prepare transforms standard to MNIST
        transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
        # data
        mnist_train = MNIST(os.getcwd(), train=True, download=True, transform=transform)
        return DataLoader(mnist_train, batch_size=64)

    def val_dataloader(self):
        transforms = ...
        mnist_val = ...
        return DataLoader(mnist_val, batch_size=64)

    def test_dataloader(self):
        transforms = ...
        mnist_test = ...
        return DataLoader(mnist_test, batch_size=64)

time: 767 µs (started: 2021-08-31 23:59:54 +08:00)


DataLoaders 已经在模型中，不需要在 .fit() 上指定。

In [ ]:
model = LitMNIST()
trainer = Trainer()
trainer.fit(model)

3. DataModules（推荐）

定义自由浮动的数据加载器、拆分、下载指令等可能会变得混乱。 在这种情况下，最好将数据集的完整定义分组到一个 DataModule 中，其中包括：
* 下载说明
* 加工说明
* 拆分说明
* 训练数据加载器
* Val 数据加载器
* 测试数据加载器

In [ ]:
class MyDataModule(LightningDataModule):
    def __init__(self):
        super().__init__()
        self.train_dims = None
        self.vocab_size = 0

    def prepare_data(self):
        # 仅在 1 个 GPU 上调用
        download_dataset()
        tokenize()
        build_vocab()

    def setup(self, stage: Optional[str] = None):
        # 在每个 GPU 上调用
        vocab = load_vocab()
        self.vocab_size = len(vocab)

        self.train, self.val, self.test = load_datasets()
        self.train_dims = self.train.next_batch.size()

    def train_dataloader(self):
        transforms = ...
        return DataLoader(self.train, batch_size=64)

    def val_dataloader(self):
        transforms = ...
        return DataLoader(self.val, batch_size=64)

    def test_dataloader(self):
        transforms = ...
        return DataLoader(self.test, batch_size=64)

使用 DataModules 可以更轻松地共享完整的数据集定义。

In [ ]:
# 使用 MNIST 数据集
mnist_dm = MNISTDatamodule()
model = LitModel(num_classes=mnist_dm.num_classes)
trainer.fit(model, mnist_dm)

# 或具有相同模型的其他数据集
imagenet_dm = ImagenetDatamodule()
model = LitModel(num_classes=imagenet_dm.num_classes)
trainer.fit(model, imagenet_dm)

### 由数据定义的模型

当您的模型需要了解数据时，最好在将数据传递给模型之前对其进行处理。

In [ ]:
# init dm 并手动调用处理
dm = ImagenetDataModule()
dm.prepare_data()
dm.setup()

model = LitModel(out_features=dm.num_classes, img_width=dm.img_width, img_height=dm.img_height)
trainer.fit(model, dm)

* 使用 prepare_data() 下载和处理数据集。
* 使用 setup() 进行拆分，并构建模型内部。

使用 DataModule 的替代方法是将模型模块的初始化推迟到 LightningModule 的`setup`方法，如下所示：

In [ ]:
class LitMNIST(LightningModule):
    def __init__(self):
        self.l1 = None

    def prepare_data(self):
        download_data()
        tokenize()

    def setup(self, stage: Optional[str] = None):
        # 阶段是“fit”、“验证”、“测试”或“预测”。 90% 的时间不相关
        data = load_data()
        num_classes = data.classes
        self.l1 = nn.Linear(..., num_classes)

### 优化器

接下来我们选择使用什么优化器来训练我们的系统。 在 PyTorch 中，我们这样做：

In [15]:
from torch.optim import Adam

optimizer = Adam(LitMNIST().parameters(), lr=1e-3)

time: 6.12 ms (started: 2021-09-01 00:18:45 +08:00)


在 Lightning 中，我们做同样的事情，但在 configure_optimizers() 方法下组织它。

In [ ]:
class LitMNIST(LightningModule):
    def configure_optimizers(self):
        return Adam(self.parameters(), lr=1e-3)

> LightningModule 本身有参数，所以传入 self.parameters()

但是，如果您有多个优化器，请使用匹配参数

In [ ]:
class LitMNIST(LightningModule):
    def configure_optimizers(self):
        return Adam(self.generator(), lr=1e-3), Adam(self.discriminator(), lr=1e-3)

### 训练步骤

训练步骤是在训练循环中发生的事情。

In [ ]:
for epoch in epochs:
    for batch in data:
        # TRAINING STEP
        # ....
        # TRAINING STEP
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

在 MNIST 的情况下，我们执行以下操作

In [ ]:
for epoch in epochs:
    for batch in data:
        # ------ TRAINING STEP START ------
        x, y = batch
        logits = model(x)
        loss = F.nll_loss(logits, y)
        # ------ TRAINING STEP END ------

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

在 Lightning 中，训练步骤中的所有内容都在 LightningModule 中的 training_step() 函数下组织。

In [ ]:
class LitMNIST(LightningModule):
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        return loss

同样，这是相同的 PyTorch 代码，只是它是由 LightningModule 组织的。 此代码不受限制，这意味着它可以像完整的 seq-2-seq、RL 循环、GAN 等一样复杂……

## 工程代码

### 训练

到目前为止，我们在纯 PyTorch 中定义了 4 个关键成分，但使用 LightningModule 组织代码。
* 模型。
* 训练数据。
* 优化器。
* 训练循环中会发生什么。

为清楚起见，我们会记得完整的 LightningModule 现在看起来像这样。

In [14]:
class LitMNIST(LightningModule):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(28 * 28, 128)
        self.layer_2 = nn.Linear(128, 256)
        self.layer_3 = nn.Linear(256, 10)

    def forward(self, x):
        batch_size, channels, height, width = x.size()
        x = x.view(batch_size, -1)
        x = self.layer_1(x)
        x = F.relu(x)
        x = self.layer_2(x)
        x = F.relu(x)
        x = self.layer_3(x)
        x = F.log_softmax(x, dim=1)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        return loss
    
    def configure_optimizers(self):
        return Adam(self.parameters(), lr=1e-3)

time: 1.08 ms (started: 2021-09-01 00:18:42 +08:00)


同样，这是相同的 PyTorch 代码，只是它由 LightningModule 组织。

### 日志记录

要登录 Tensorboard、您最喜欢的记录器和/或进度条，请使用 log() 方法，该方法可以从 LightningModule 中的任何方法调用。

In [ ]:
def training_step(self, batch, batch_idx):
    self.log("my_metric", x)

log() 方法有几个选项：
* on_step（记录训练中该步骤的指标）
* on_epoch（在epoch结束时自动累积和记录）
* prog_bar（记录到进度条）
* logger（像 Tensorboard 一样记录到记录器）

根据调用日志的位置，Lightning 会自动为您确定正确的模式。 但是当然您可以通过手动设置标志来覆盖默认行为。

> 设置 on_epoch=True 将在整个训练时期累积您的记录值。

In [ ]:
def training_step(self, batch, batch_idx):
    self.log("my_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)

您还可以直接使用记录器的任何方法：

In [ ]:
def training_step(self, batch, batch_idx):
    tensorboard = self.logger.experiment
    tensorboard.any_summary_writer_method_you_want()

训练开始后，您可以使用自己喜欢的记录器或启动 Tensorboard 日志来查看日志：

In [ ]:
%tensorboard --logdir ./lightning_logs

这将生成自动tensorboard日志（或使用您选择的记录器）。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gu0bxe6t3vj612e0nktaw02.jpg)

但是您也可以使用我们支持的任何数量的其他记录器。

### 在 CPU 上训练

In [17]:
from pytorch_lightning import Trainer

model = LitMNIST()
trainer = Trainer(progress_bar_refresh_rate = 1)
trainer.fit(model, mnist_train)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/data/miniconda3/envs/torch19/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1292: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  rank_zero_warn(

  | Name    | Type   | Params
-----------------------------------
0 | layer_1 | Linear | 100 K 
1 | layer_2 | Linear | 33.0 K
2 | layer_3 | Linear | 2.6 K 
-----------------------------------
136 K     Trainable params
0         Non-trainable params
136 K     Total params
0.544     Total estimated model params size (MB)


Training: -1it [00:00, ?it/s]

/data/miniconda3/envs/torch19/lib/python3.9/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 48 worker processes in total. Our suggested max number of worker in current system is 40, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


time: 33.1 s (started: 2021-09-01 00:18:57 +08:00)


/data/miniconda3/envs/torch19/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1047: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


### 在 GPU 上训练

但最好的地方是你可以用trainer flag做的所有魔法。 例如，要在 GPU 上运行此模型：

In [25]:
model = LitMNIST()
trainer = Trainer(gpus=1)
trainer.fit(model, mnist_train)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7,8,9]

  | Name    | Type   | Params
-----------------------------------
0 | layer_1 | Linear | 100 K 
1 | layer_2 | Linear | 33.0 K
2 | layer_3 | Linear | 2.6 K 
-----------------------------------
136 K     Trainable params
0         Non-trainable params
136 K     Total params
0.544     Total estimated model params size (MB)


Training: -1it [00:00, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter serve

RuntimeError: DataLoader worker (pid(s) 2312129, 2312169, 2312209, 2312249, 2312289, 2312329, 2312369, 2312409, 2312449, 2312489, 2312529, 2312569, 2312609, 2312649, 2312689, 2312729, 2312769, 2312809, 2312849, 2312889, 2312929, 2312969) exited unexpectedly

time: 1h 59min 52s (started: 2021-09-01 00:39:58 +08:00)


### 在多 GPU 上训练

或者，您也可以在多个 GPU 上进行训练。

In [24]:
model = LitMNIST()
trainer = Trainer(gpus=8, accelerator='dp')
trainer.fit(model, mnist_train)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7,8,9]

  | Name    | Type   | Params
-----------------------------------
0 | layer_1 | Linear | 100 K 
1 | layer_2 | Linear | 33.0 K
2 | layer_3 | Linear | 2.6 K 
-----------------------------------
136 K     Trainable params
0         Non-trainable params
136 K     Total params
0.544     Total estimated model params size (MB)


Training: -1it [00:00, ?it/s]

time: 8.97 s (started: 2021-09-01 00:39:42 +08:00)


/data/miniconda3/envs/torch19/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1047: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


或者多个节点

In [ ]:
# (32 GPUs)
model = LitMNIST()
trainer = Trainer(gpus=8, num_nodes=4, accelerator="ddp")
trainer.fit(model, train_loader)

## 超参数

Lightning 具有与命令行 ArgumentParser 无缝交互的实用程序，并与您选择的超参数优化框架配合得很好。

### 参数解析器

Lightning 旨在增强内置 Python ArgumentParser 的许多功能

In [ ]:
from argparse import ArgumentParser

parser = ArgumentParser()
parser.add_argument("--layer_1_dim", type=int, default=128)
args = parser.parse_args()

这允许您像这样调用您的程序：

In [ ]:
python trainer.py --layer_1_dim 64

### Argparser 最佳实践

最佳做法是将您的论点分为三个部分。
1. 训练器参数（gpus、num_nodes 等...）
2. 模型特定参数（layer_dim、num_layers、learning_rate 等...）
3. 程序参数（data_path、cluster_email 等...）

我们可以这样做。 首先，在您的 LightningModule 中，定义特定于该模块的参数。 请记住，数据拆分或数据路径也可能特定于某个模块（即：如果您的项目有一个在 Imagenet 上训练的模型和另一个在 CIFAR-10 上训练的模型）。

In [ ]:
class LitModel(LightningModule):
    @staticmethod
    def add_model_specific_args(parent_parser):
        parser = parent_parser.add_argument_group("LitModel")
        parser.add_argument("--encoder_layers", type=int, default=12)
        parser.add_argument("--data_path", type=str, default="/some/path")
        return parent_parser

现在，在您的主训练文件中，添加 Trainer args、程序 args，并添加模型 args

In [ ]:
# ----------------
# trainer_main.py
# ----------------
from argparse import ArgumentParser

parser = ArgumentParser()

# 添加程序级参数
parser.add_argument("--conda_env", type=str, default="some_name")
parser.add_argument("--notification_email", type=str, default="will@email.com")

# 添加特定于模型的参数
parser = LitModel.add_model_specific_args(parser)

# 将所有可用的训练器选项添加到 argparse
# 即：现在 --gpus --num_nodes ... --fast_dev_run 在 cli 中的所有工作
parser = Trainer.add_argparse_args(parser)

args = parser.parse_args()

现在你可以像这样调用运行你的程序：

In [ ]:
python trainer_main.py --gpus 2 --num_nodes 2 --conda_env 'my_env' --encoder_layers 12

最后，确保像这样开始训练：

In [ ]:
# 像这样初始化训练器
trainer = Trainer.from_argparse_args(args, early_stopping_callback=...)

# 不喜欢这个
trainer = Trainer(gpus=hparams.gpus, ...)

# 直接用 Namespace 初始化模型
model = LitModel(args)

# 或使用所有键值对初始化模型
dict_args = vars(args)
model = LitModel(**dict_args)

## LightningModule 超参数

很多时候，我们训练一个模型的多个版本。 您可能会分享该模型或在几个月后再次使用该模型，此时了解该模型是如何训练的（即：什么学习率、神经网络等）非常有用。

Lightning 有几种方法可以将这些信息保存在检查点和 yaml 文件中。 这里的目标是提高可读性和再现性。

1. 在 LightningModule `__init__` 函数中使用 `save_hyperparameters()` 将使 Lightning 能够将所有提供的参数存储在 `self.hparams` 属性中。 这些超参数也将存储在模型检查点中，这简化了生产设置中的模型重新实例化。 这也使这些值可以通过 `self.hparams`获得。

In [ ]:
class LitMNIST(LightningModule):
    def __init__(self, layer_1_dim=128, learning_rate=1e-2, **kwargs):
        super().__init__()
        # 调用它以将 (layer_1_dim=128, learning_rate=1e-4) 保存到检查点
        self.save_hyperparameters()

        # 相等的
        self.save_hyperparameters("layer_1_dim", "learning_rate")

        # 现在可以从 hparams 访问 layer_1_dim
        self.hparams.layer_1_dim

2. 有时，您的 init 可能具有您可能不想保存的对象或其他参数。 在这种情况下，只选择几个

In [ ]:
class LitMNIST(LightningModule):
    def __init__(self, loss_fx, generator_network, layer_1_dim=128 ** kwargs):
        super().__init__()
        self.layer_1_dim = layer_1_dim
        self.loss_fx = loss_fx

        # 调用它以将 (layer_1_dim=128) 保存到检查点
        self.save_hyperparameters("layer_1_dim")


#加载指定其他参数
model = LitMNIST.load_from_checkpoint(PATH, loss_fx=torch.nn.SomeOtherLoss, generator_network=MyGenerator())

3. 您还可以将 dict 或 Namespace 等完整对象转换为 hparams，以便将它们保存到检查点。

In [ ]:
class LitMNIST(LightningModule):
    def __init__(self, conf: Optional[Union[Dict, Namespace, DictConfig]] = None, **kwargs):
        super().__init__()
        # save the config and any extra arguments
        self.save_hyperparameters(conf)
        self.save_hyperparameters(kwargs)

        self.layer_1 = nn.Linear(28 * 28, self.hparams.layer_1_dim)
        self.layer_2 = nn.Linear(self.hparams.layer_1_dim, self.hparams.layer_2_dim)
        self.layer_3 = nn.Linear(self.hparams.layer_2_dim, 10)


conf = {...}
# OR
# conf = parser.parse_args()
# OR
# conf = OmegaConf.create(...)
model = LitMNIST(conf=conf, anything=10)

# 现在可以从 hparams 访问任何存储的变量
model.hparams.anything

# 为此，您需要使用 `self.hparams.layer_1_dim` 访问，而不是 `conf.layer_1_dim`
model = LitMNIST.load_from_checkpoint(PATH)

### 训练器参数

回顾一下，将所有可能的训练器标志添加到 argparser 并以这种方式初始化训练器

In [ ]:
parser = ArgumentParser()
parser = Trainer.add_argparse_args(parser)
hparams = parser.parse_args()

trainer = Trainer.from_argparse_args(hparams)

#或者如果您需要传入回调
trainer = Trainer.from_argparse_args(hparams, checkpoint_callback=..., callbacks=[...])

### 多个闪电模块

我们经常有多个闪电模块，每个模块都有不同的参数。 LightningModule 允许您为每个文件定义参数，而不是污染 main.py 文件。

In [ ]:
class LitMNIST(LightningModule):
    def __init__(self, layer_1_dim, **kwargs):
        super().__init__()
        self.layer_1 = nn.Linear(28 * 28, layer_1_dim)

    @staticmethod
    def add_model_specific_args(parent_parser):
        parser = parent_parser.add_argument_group("LitMNIST")
        parser.add_argument("--layer_1_dim", type=int, default=128)
        return parent_parser

In [ ]:
class GoodGAN(LightningModule):
    def __init__(self, encoder_layers, **kwargs):
        super().__init__()
        self.encoder = Encoder(layers=encoder_layers)

    @staticmethod
    def add_model_specific_args(parent_parser):
        parser = parent_parser.add_argument_group("GoodGAN")
        parser.add_argument("--encoder_layers", type=int, default=12)
        return parent_parser

现在我们可以允许每个模型在 main.py 中注入它需要的参数

In [ ]:
def main(args):
    dict_args = vars(args)

    # pick model
    if args.model_name == "gan":
        model = GoodGAN(**dict_args)
    elif args.model_name == "mnist":
        model = LitMNIST(**dict_args)

    trainer = Trainer.from_argparse_args(args)
    trainer.fit(model)


if __name__ == "__main__":
    parser = ArgumentParser()
    parser = Trainer.add_argparse_args(parser)

    # figure out which model to use
    parser.add_argument("--model_name", type=str, default="gan", help="gan or mnist")

    # THIS LINE IS KEY TO PULL THE MODEL NAME
    temp_args, _ = parser.parse_known_args()

    # let the model add what it wants
    if temp_args.model_name == "gan":
        parser = GoodGAN.add_model_specific_args(parser)
    elif temp_args.model_name == "mnist":
        parser = LitMNIST.add_model_specific_args(parser)

    args = parser.parse_args()

    # train
    main(args)

现在我们可以使用命令行界面训练 MNIST 或 GAN！

In [ ]:
$ python main.py --model_name gan --encoder_layers 24
$ python main.py --model_name mnist --layer_1_dim 128

## Validating

在大多数情况下，当数据验证拆分的性能达到最小值时，我们会停止训练模型。

就像 training_step 一样，我们可以定义一个 validation_step 来检查我们关心的任何指标、生成样本或向日志中添加更多指标。

In [ ]:
def validation_step(self, batch, batch_idx):
    loss = MSE_loss(...)
    self.log("val_loss", loss)

现在我们也可以使用验证循环进行训练。

In [ ]:
from pytorch_lightning import Trainer

model = LitMNIST()
trainer = Trainer(tpu_cores=8)
trainer.fit(model, train_loader, val_loader)

您可能已经注意到记录的“验证健全性检查”字样。 这是因为 Lightning 在开始训练之前运行了 2 批验证。 这是一种单元测试，以确保如果您在验证循环中有错误，您无需等待一个完整的 epoch 才能找出答案。

> Lightning 禁用梯度，将模型置于评估模式，并执行验证所需的一切。

### 引擎盖下的 Val 循环

在幕后，Lightning 执行以下操作：

In [ ]:
model = Model()
model.train()
torch.set_grad_enabled(True)

for epoch in epochs:
    for batch in data:
        # train
        ...

    # validate
    model.eval()
    torch.set_grad_enabled(False)

    outputs = []
    for batch in val_data:
        x, y = batch  # validation_step
        y_hat = model(x)  # validation_step
        loss = loss(y_hat, x)  # validation_step
        outputs.append({"val_loss": loss})  # validation_step

    total_loss = outputs.mean()  # validation_epoch_end

### 可选方法

如果您仍然需要更细粒度的控制，请为循环定义其他可选方法。

In [ ]:
def validation_step(self, batch, batch_idx):
    preds = ...
    return preds


def validation_epoch_end(self, val_step_outputs):
    for pred in val_step_outputs:
        # do something with all the predictions from each validation_step
        ...

## 测试

一旦我们的研究完成并且我们即将发布或部署一个模型，我们通常想弄清楚它如何在“现实世界”中推广。 为此，我们使用保留的数据拆分进行测试。

就像验证循环一样，我们定义了一个测试循环

In [ ]:
class LitMNIST(LightningModule):
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        self.log("test_loss", loss)

但是，为了确保不会无意中使用测试集，Lightning 有一个单独的 API 来运行测试。 训练模型后，只需调用 .test()。

In [ ]:
from pytorch_lightning import Trainer

model = LitMNIST()
trainer = Trainer(tpu_cores=8)
trainer.fit(model)

# run test set
result = trainer.test()
print(result)

您还可以从保存的闪电模型运行测试

In [ ]:
model = LitMNIST.load_from_checkpoint(PATH)
trainer = Trainer(tpu_cores=8)
trainer.test(model)

## 预测

同样，LightningModule 与 PyTorch 模块完全相同。 这意味着您可以加载它并将其用于预测。

In [ ]:
model = LitMNIST.load_from_checkpoint(PATH)
x = torch.randn(1, 1, 28, 28)
out = model(x)

从表面上看，forward 和 training_step 是类似的。 通常，我们要确保我们希望模型做的事情是在前进中发生的事情。 而 training_step 可能从它内部调用。

In [ ]:
class MNISTClassifier(LightningModule):
    def forward(self, x):
        batch_size, channels, height, width = x.size()
        x = x.view(batch_size, -1)
        x = self.layer_1(x)
        x = F.relu(x)
        x = self.layer_2(x)
        x = F.relu(x)
        x = self.layer_3(x)
        x = F.log_softmax(x, dim=1)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        return loss

In [ ]:
model = MNISTClassifier()
x = mnist_image()
logits = model(x)

在这种情况下，我们设置了这个 LightningModel 来预测 logits。 但我们也可以让它预测特征图：

In [ ]:
class MNISTRepresentator(LightningModule):
    def forward(self, x):
        batch_size, channels, height, width = x.size()
        x = x.view(batch_size, -1)
        x = self.layer_1(x)
        x1 = F.relu(x)
        x = self.layer_2(x1)
        x2 = F.relu(x)
        x3 = self.layer_3(x2)
        return [x, x1, x2, x3]

    def training_step(self, batch, batch_idx):
        x, y = batch
        out, l1_feats, l2_feats, l3_feats = self(x)
        logits = F.log_softmax(out, dim=1)
        ce_loss = F.nll_loss(logits, y)
        loss = perceptual_loss(l1_feats, l2_feats, l3_feats) + ce_loss
        return loss

In [ ]:
model = MNISTRepresentator.load_from_checkpoint(PATH)
x = mnist_image()
feature_maps = model(x)

或者也许我们有一个用于生成的模型。 LightningModule 也只是一个 torch.nn.Module。

In [ ]:
class LitMNISTDreamer(LightningModule):
    def forward(self, z):
        imgs = self.decoder(z)
        return imgs

    def training_step(self, batch, batch_idx):
        x, y = batch
        representation = self.encoder(x)
        imgs = self(representation)

        loss = perceptual_loss(imgs, x)
        return loss

In [ ]:
model = LitMNISTDreamer.load_from_checkpoint(PATH)
z = sample_noise()
generated_imgs = model(z)

要大规模执行推理，可以使用 predict() 和 predict_step() 默认情况下，predict_step() 调用 forward()，但可以覆盖它以添加任何处理逻辑。

In [ ]:
class LitMNISTDreamer(LightningModule):
    def forward(self, z):
        imgs = self.decoder(z)
        return imgs

    def predict_step(self, batch, batch_idx: int, dataloader_idx: int = None):
        return self(batch)


model = LitMNISTDreamer()
trainer.predict(model, datamodule)

您如何拆分 forward() 与 training_step() 与 predict_step() 中的内容取决于您希望如何使用此模型进行预测。 但是，我们建议 forward() 仅包含您的模型的张量操作。 training_step() 用日志记录、指标和损失计算封装 forward() 逻辑。 predict_step() 用任何必要的预处理或后处理函数封装 forward()。

## 非关键环节

### 可扩展性

尽管闪电让一切变得超级简单，但它并没有牺牲任何灵活性或控制力。 Lightning 提供了多种管理训练状态的方法。

### 训练覆盖

训练、验证和测试循环的任何部分都可以修改。 例如，如果您想进行自己的向后传递，则可以覆盖默认实现

In [ ]:
def backward(self, use_amp, loss, optimizer):
    loss.backward()

用你自己的

In [ ]:
class LitMNIST(LightningModule):
    def backward(self, use_amp, loss, optimizer, optimizer_idx):
        # do a custom way of backward
        loss.backward(retain_graph=True)

训练的每个部分都可以通过这种方式进行配置。 有关完整列表，请查看 LightningModule。

### 回调

添加任意功能的另一种方法是为您可能关心的钩子添加自定义回调

In [ ]:
from pytorch_lightning.callbacks import Callback


class MyPrintingCallback(Callback):
    def on_init_start(self, trainer):
        print("Starting to init trainer!")

    def on_init_end(self, trainer):
        print("Trainer is init now")

    def on_train_end(self, trainer, pl_module):
        print("do something when training ends")

并将回调传递给训练器

### 子模块

研究项目倾向于测试针对同一数据集的不同方法。 这在带有继承的 Lightning 中很容易做到。

例如，假设我们现在想要训练一个自动编码器用作 MNIST 图像的特征提取器。 我们正在从 LitMNIST 模块扩展我们的自动编码器，该模块已经定义了所有数据加载。 Autoencoder 模型中唯一改变的是初始化、转发、训练、验证和测试步骤。

In [6]:
class Encoder(torch.nn.Module):
    pass


class Decoder(torch.nn.Module):
    pass


class AutoEncoder(LitMNIST):
    def __init__(self):
        super().__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()
        self.metric = MSE()

    def forward(self, x):
        return self.encoder(x)

    def training_step(self, batch, batch_idx):
        x, _ = batch

        representation = self.encoder(x)
        x_hat = self.decoder(representation)

        loss = self.metric(x, x_hat)
        return loss

    def validation_step(self, batch, batch_idx):
        self._shared_eval(batch, batch_idx, "val")

    def test_step(self, batch, batch_idx):
        self._shared_eval(batch, batch_idx, "test")

    def _shared_eval(self, batch, batch_idx, prefix):
        x, _ = batch
        representation = self.encoder(x)
        x_hat = self.decoder(representation)

        loss = self.metric(x, x_hat)
        self.log(f"{prefix}_loss", loss)

time: 1.7 ms (started: 2021-09-01 13:27:38 +08:00)


我们可以使用同一个训练器来训练它

In [ ]:
autoencoder = AutoEncoder()
trainer = Trainer(gpus=1, max_epochs=10)
trainer.fit(autoencoder)

请记住，forward 方法应该定义 LightningModule 的实际用途。 在这种情况下，我们想使用 AutoEncoder 来提取图像表示

In [ ]:
some_images = torch.Tensor(32, 1, 28, 28)
representations = autoencoder(some_images)

## 迁移学习

### 使用预训练模型

有时我们想使用 LightningModule 作为预训练模型。 这很好，因为 LightningModule 只是一个 torch.nn.Module！

让我们在单独的模型中使用 AutoEncoder 作为特征提取器。

class Encoder(torch.nn.Module):
    ...


class AutoEncoder(LightningModule):
    def __init__(self):
        self.encoder = Encoder()
        self.decoder = Decoder()


class CIFAR10Classifier(LightningModule):
    def __init__(self):
        # init the pretrained LightningModule
        self.feature_extractor = AutoEncoder.load_from_checkpoint(PATH)
        self.feature_extractor.freeze()

        # the autoencoder outputs a 100-dim representation and CIFAR-10 has 10 classes
        self.classifier = nn.Linear(100, 10)

    def forward(self, x):
        representations = self.feature_extractor(x)
        x = self.classifier(representations)
        ...

我们使用预训练的自动编码器（LightningModule）进行迁移学习！

示例：Imagenet（计算机视觉）

In [ ]:
import torchvision.models as models


class ImagenetTransferLearning(LightningModule):
    def __init__(self):
        super().__init__()

        # init a pretrained resnet
        backbone = models.resnet50(pretrained=True)
        num_filters = backbone.fc.in_features
        layers = list(backbone.children())[:-1]
        self.feature_extractor = nn.Sequential(*layers)

        # use the pretrained model to classify cifar-10 (10 image classes)
        num_target_classes = 10
        self.classifier = nn.Linear(num_filters, num_target_classes)

    def forward(self, x):
        self.feature_extractor.eval()
        with torch.no_grad():
            representations = self.feature_extractor(x).flatten(1)
        x = self.classifier(representations)
        ...

微调

In [ ]:
model = ImagenetTransferLearning()
trainer = Trainer()
trainer.fit(model)

并用它来预测您感兴趣的数据

In [ ]:
model = ImagenetTransferLearning.load_from_checkpoint(PATH)
model.freeze()

x = some_images_from_cifar10()
predictions = model(x)

我们在 imagenet 上使用了预训练模型，在 CIFAR-10 上进行了微调，以在 CIFAR-10 上进行预测。 在非学术界，我们会在你拥有的一个小数据集上进行微调，并在你的数据集上进行预测。

示例：BERT (NLP)

只要它是 torch.nn.Module 子类，Lightning 与用于迁移学习的内容完全无关。

这是一个使用 Huggingface Transformer 的模型。

In [ ]:
class BertMNLIFinetuner(LightningModule):
    def __init__(self):
        super().__init__()

        self.bert = BertModel.from_pretrained("bert-base-cased", output_attentions=True)
        self.W = nn.Linear(bert.config.hidden_size, 3)
        self.num_classes = 3

    def forward(self, input_ids, attention_mask, token_type_ids):

        h, _, attn = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        h_cls = h[:, 0]
        logits = self.W(h_cls)
        return logits, attn

## 为什么使用 PyTorch 闪电

1. 更少的样板

研究和生产代码从简单的代码开始，但一旦添加 GPU 训练、16 位、检查点、日志记录等，复杂性就会迅速增加……

PyTorch Lightning 为您实现了这些功能，并对其进行了严格的测试，以确保您可以专注于研究想法。

编写更少的工程/样板代码意味着：
* 更少的错误
* 更快的迭代
* 更快的原型设计

2. 更多功能

在 PyTorch Lightning 中，您可以利用来自世界顶级 AI 实验室的数百名 AI 研究人员、研究工程师和博士编写的代码，实现所有最新的最佳实践和 SOTA 功能，例如
* GPU、多 GPU、TPU 训练
* 多节点训练
* 自动记录
* …
* 梯度积累

3. 不易出错

为什么要重新发明轮子？

使用 PyTorch Lightning 享受深度学习结构，该结构在每次拉取请求时都经过 CPU/多 GPU/多 TPU 的严格测试（500 多次测试）。

我们承诺，我们来自顶级实验室的 20 多名集体团队比您更考虑培训:)

4. 不是新库

PyTorch Lightning 是有组织的 PyTorch - 无需学习新框架。

在此处了解如何从 PyTorch 转换为 Lightning。

您的项目将变得越来越复杂，最终您将更多地进行工程而不是尝试新想法……将最困难的部分推迟到 Lightning！

## Lightning哲学

Lightning 将您的深度学习代码分为 4 部分：
* 研究代码
* 工程代码
* 非关键代码
* 数据代码

### 研究代码

在 MNIST 生成示例中，研究代码将是特定系统及其训练方式（即：GAN 或 VAE 或 GPT）。

In [ ]:
l1 = nn.Linear(...)
l2 = nn.Linear(...)
decoder = Decoder()

x1 = l1(x)
x2 = l2(x2)
out = decoder(features, x)

loss = perceptual_loss(x1, x2, x) + CE(out, x)

在 Lightning 中，此代码被组织成一个 Lightning 模块。

### 工程代码

工程代码是与训练该系统相关的所有代码。 诸如提前停止、GPU 分布、16 位精度等。这通常是大多数项目中相同的代码。

In [ ]:
model.cuda(0)
x = x.cuda(0)

distributed = DistributedParallel(model)

with gpu_zero:
    download_data()

dist.barrier()

在 Lightning 中，这段代码是由训练器抽象出来的。

### 非关键代码

这是有助于研究但与研究代码无关的代码。 一些例子可能是：
* 检查梯度
* 登录到tensorboard。

In [ ]:
# log samples
z = Q.rsample()
generated = decoder(z)
self.experiment.log("images", generated)

### 数据代码

Lightning 使用标准的 PyTorch DataLoaders 或任何提供一批数据的东西。 这段代码最终会因转换、规范化常量和分布在整个文件中的数据拆分而变得混乱。

In [ ]:
# data
train = MNIST(...)
train, val = split(train, val)
test = MNIST(...)

# transforms
train_transforms = ...
val_transforms = ...
test_transforms = ...

# dataloader ...
# download with dist.barrier() for multi-gpu, etc...

一旦您开始进行多 GPU 训练或需要有关数据的信息来构建模型，此代码就会变得特别复杂。

在 Lightning 中，此代码组织在数据模块内。